In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

pd.set_option('max_column', 100)

In [2]:
#Function to replace spaces with underscores and removing \xa0
def clean_col(c):
    return c.lower().strip().replace(" ", "_").replace("\xa0", "")

In [3]:
raw_path = '../../datasets/raw/'
mst_path = '../../datasets/mst/'

In [4]:
os.listdir(raw_path)

['ParentSurveyEnglish.csv',
 'ParentSurveySpanish.xlsx',
 'StudentSurveyEnglish.csv']

In [22]:
df = pd.read_csv(raw_path + 'StudentSurveyEnglish.csv')
df.head(2)

,Respondent ID,Collector ID,Start Date,End Date,IP Address,Email Address,First Name,Last Name,Custom Data 1,What grade are you in?,Unnamed: 10,How did you got to school this morning? (Check all that apply),Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Who brought you to school this morning? (Check all that apply),Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Do you think walking and biking to school is? (Check all that apply,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Have you ever heard of the following? (Check all that apply),Unnamed: 30,Unnamed: 31,Unnamed: 32
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Response,Other (please specify),Walking,Car,Bus,Metro Rail (train),Bike,Scooter,Other (please specify),Parent/Grandparent,Neighbor,Friend/s,Came alone,Other (please specify),Fun,Healthy,Good for me,Scary,Tiring,Takes too long,"""Stop, look, listen""","""Always wear a helmet""","""Left, right left again""","""Scan, signal, scan"""
1,6.950670e+09,173176001.0,10/18/2018 03:57:57 PM,10/18/2018 03:58:33 PM,74.62.204.86,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,Bus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,Good for me,NaN,NaN,NaN,"""Stop, look, listen""","""Always wear a helmet""","""Left, right left again""",NaN


In [23]:
# Get the column names, the headers that contains "unnamed", and values of the second row and their corresponding indice
col_names = list(df.columns)
unnameds = [x for x in col_names if 'Unnamed' in x]
sec_row = df.loc[0, ].values
inds = [col_names.index(x) for x in unnameds]

print(col_names)

# Add the indices of columns that has Unnamed in there
temp = []

# Add in the first and last indice by hand
temp.append(9)

for i in range(len(inds) - 1):
    value = inds[i]
    next_value = inds[i+1]
    temp.append(value)
    if value < next_value - 1:
        temp.append(next_value-1)  
temp.append(next_value)

# Replace column names by second row values whenever needed     
for i in range(len(col_names)):
    if i in temp:
        col_names[i] = sec_row[i]
        
print(temp)

['Respondent ID', 'Collector ID', 'Start Date', 'End Date', 'IP Address', 'Email Address', 'First Name', 'Last Name', 'Custom Data 1', 'What grade are you in?', 'Unnamed: 10', 'How did you got to school this morning? (Check all that apply)', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Who brought you to school this morning? (Check all that apply)', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Do you think walking and biking to school is? (Check all that apply', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Have you ever heard of the following? (Check all that apply)', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32']
[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]


In [24]:
# Create a question look-up table
q_map = defaultdict(list)

# retrieve questions that span multiple columns
q_list = [c for c in df.columns if c not in col_names]

print(q_list)
question = q_list[0]
for i in range(len(q_list)):
    if 'Unname' in q_list[i]:
        q_list[i] = question
    else:
        question = q_list[i]

print(q_list)

# make a list of the responses to the multi-column questions
for i in temp:
    q_map['ans_choice'].append(sec_row[i])

# put the two parts, q_list and q_map together
q_map['question'] = q_list
q_map = pd.DataFrame(q_map)
q_map

['What grade are you in?', 'Unnamed: 10', 'How did you got to school this morning? (Check all that apply)', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Who brought you to school this morning? (Check all that apply)', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Do you think walking and biking to school is? (Check all that apply', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Have you ever heard of the following? (Check all that apply)', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32']
['What grade are you in?', 'What grade are you in?', 'How did you got to school this morning? (Check all that apply)', 'How did you got to school this morning? (Check all that apply)', 'How did you got to school this morning? (Check all that apply)', 'How did you got to school this morning? (Check all that apply)', 'How did you got to school this morning? (Check all that apply)', 'How did you got to school this mornin

,ans_choice,question
0,Response,What grade are you in?
1,Other (please specify),What grade are you in?
2,Walking,How did you got to school this morning? (Check...
3,Car,How did you got to school this morning? (Check...
4,Bus,How did you got to school this morning? (Check...
5,Metro Rail (train),How did you got to school this morning? (Check...
6,Bike,How did you got to school this morning? (Check...
7,Scooter,How did you got to school this morning? (Check...
8,Other (please specify),How did you got to school this morning? (Check...
9,Parent/Grandparent,Who brought you to school this morning? (Check...


In [25]:
# saving q_map to a csv file
q_map.to_csv(mst_path + 'question_lookup.csv', index = None)
q_map = pd.read_csv(mst_path + 'question_lookup.csv')
q_map.head()

,ans_choice,question
0,Response,What grade are you in?
1,Other (please specify),What grade are you in?
2,Walking,How did you got to school this morning? (Check...
3,Car,How did you got to school this morning? (Check...
4,Bus,How did you got to school this morning? (Check...


In [26]:
# Replace columns names 
col_names = [clean_col(c) for c in col_names]
df.columns = col_names

df
# Delete second row
df=df.drop(df.index[0])

In [27]:
df.head()

,respondent_id,collector_id,start_date,end_date,ip_address,email_address,first_name,last_name,custom_data_1,response,other_(please_specify),walking,car,bus,metro_rail_(train),bike,scooter,other_(please_specify),parent/grandparent,neighbor,friend/s,came_alone,other_(please_specify),fun,healthy,good_for_me,scary,tiring,takes_too_long,"""stop,_look,_listen""","""always_wear_a_helmet""","""left,_right_left_again""","""scan,_signal,_scan"""
1,6.950670e+09,173176001.0,10/18/2018 03:57:57 PM,10/18/2018 03:58:33 PM,74.62.204.86,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,Bus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,Good for me,NaN,NaN,NaN,"""Stop, look, listen""","""Always wear a helmet""","""Left, right left again""",NaN
2,6.950669e+09,173176001.0,10/18/2018 03:57:29 PM,10/18/2018 03:57:52 PM,74.62.204.86,NaN,NaN,NaN,NaN,5,NaN,NaN,Car,NaN,NaN,NaN,NaN,NaN,Parent/Grandparent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takes too long,NaN,"""Always wear a helmet""",NaN,NaN
3,6.950669e+09,173176001.0,10/18/2018 03:57:10 PM,10/18/2018 03:57:20 PM,74.62.204.86,NaN,NaN,NaN,NaN,5,NaN,NaN,Car,NaN,NaN,NaN,NaN,NaN,Parent/Grandparent,NaN,NaN,NaN,NaN,Fun,NaN,NaN,NaN,NaN,NaN,NaN,"""Always wear a helmet""",NaN,NaN
4,6.950669e+09,173176001.0,10/18/2018 03:56:49 PM,10/18/2018 03:57:05 PM,74.62.204.86,NaN,NaN,NaN,NaN,5,NaN,NaN,Car,NaN,NaN,NaN,NaN,NaN,Parent/Grandparent,NaN,NaN,NaN,NaN,NaN,Healthy,NaN,NaN,NaN,NaN,"""Stop, look, listen""",NaN,NaN,NaN
5,6.950669e+09,173176001.0,10/18/2018 03:56:30 PM,10/18/2018 03:56:43 PM,74.62.204.86,NaN,NaN,NaN,NaN,5,NaN,NaN,Car,NaN,NaN,NaN,NaN,NaN,Parent/Grandparent,NaN,NaN,NaN,NaN,NaN,Healthy,NaN,NaN,NaN,NaN,NaN,NaN,"""Left, right left again""",NaN


In [28]:
df.to_csv(mst_path + 'StudentSurveyEnglish.csv', index = None)

In [29]:
# proportion of responses are NaN
df.isna().sum()/len(df)

respondent_id               0.000000
collector_id                0.000000
start_date                  0.000000
end_date                    0.000000
ip_address                  0.000000
email_address               1.000000
first_name                  1.000000
last_name                   1.000000
custom_data_1               1.000000
response                    0.000000
other_(please_specify)      0.995349
walking                     0.725581
car                         0.279070
bus                         0.986047
metro_rail_(train)          1.000000
bike                        0.995349
scooter                     1.000000
other_(please_specify)      0.990698
parent/grandparent          0.120930
neighbor                    0.986047
friend/s                    0.986047
came_alone                  0.930233
other_(please_specify)      0.967442
fun                         0.632558
healthy                     0.511628
good_for_me                 0.753488
scary                       0.925581
t

In [30]:
df.drop(df.columns[range(2,8)], axis=1,inplace=True)
df.columns
df.head()

,respondent_id,collector_id,custom_data_1,response,other_(please_specify),walking,car,bus,metro_rail_(train),bike,scooter,other_(please_specify),parent/grandparent,neighbor,friend/s,came_alone,other_(please_specify),fun,healthy,good_for_me,scary,tiring,takes_too_long,"""stop,_look,_listen""","""always_wear_a_helmet""","""left,_right_left_again""","""scan,_signal,_scan"""
1,6.950670e+09,173176001.0,NaN,5,NaN,NaN,NaN,Bus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,Good for me,NaN,NaN,NaN,"""Stop, look, listen""","""Always wear a helmet""","""Left, right left again""",NaN
2,6.950669e+09,173176001.0,NaN,5,NaN,NaN,Car,NaN,NaN,NaN,NaN,NaN,Parent/Grandparent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takes too long,NaN,"""Always wear a helmet""",NaN,NaN
3,6.950669e+09,173176001.0,NaN,5,NaN,NaN,Car,NaN,NaN,NaN,NaN,NaN,Parent/Grandparent,NaN,NaN,NaN,NaN,Fun,NaN,NaN,NaN,NaN,NaN,NaN,"""Always wear a helmet""",NaN,NaN
4,6.950669e+09,173176001.0,NaN,5,NaN,NaN,Car,NaN,NaN,NaN,NaN,NaN,Parent/Grandparent,NaN,NaN,NaN,NaN,NaN,Healthy,NaN,NaN,NaN,NaN,"""Stop, look, listen""",NaN,NaN,NaN
5,6.950669e+09,173176001.0,NaN,5,NaN,NaN,Car,NaN,NaN,NaN,NaN,NaN,Parent/Grandparent,NaN,NaN,NaN,NaN,NaN,Healthy,NaN,NaN,NaN,NaN,NaN,NaN,"""Left, right left again""",NaN


In [34]:
df.fillna(False, inplace=True)
for col in [c for c in df.columns[2:] if c.count('other')==0]:
    df[col] = np.where(df[col]==False, False, True)
    
df.to_csv(mst_path + 'StudentSurveyEnglish.csv', index = None)
df.head()
 

,respondent_id,collector_id,custom_data_1,response,other_(please_specify),walking,car,bus,metro_rail_(train),bike,scooter,other_(please_specify),parent/grandparent,neighbor,friend/s,came_alone,other_(please_specify),fun,healthy,good_for_me,scary,tiring,takes_too_long,"""stop,_look,_listen""","""always_wear_a_helmet""","""left,_right_left_again""","""scan,_signal,_scan"""
1,6.950670e+09,173176001.0,False,True,False,False,False,True,False,False,False,False,False,False,False,False,X,False,False,True,False,False,False,True,True,True,False
2,6.950669e+09,173176001.0,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False
3,6.950669e+09,173176001.0,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,False
4,6.950669e+09,173176001.0,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False
5,6.950669e+09,173176001.0,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False
